In [5]:
import sys
import os

# Get the absolute path of the project root
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATA_PATH=os.path.abspath(os.path.join(os.getcwd(), "../../data"))
MODEL_PATH=os.path.abspath(os.path.join(os.getcwd(), "../models"))
print(DATA_PATH)
# Add 'src/' to Python path
sys.path.append(os.path.join(PROJECT_ROOT, "src"))

# Verify the path
print("Updated sys.path:", sys.path)

/Users/pravinut/vnit/sem2/MLOPS/Assignments/Assignment2/data
Updated sys.path: ['/opt/anaconda3/envs/mlopsproj/lib/python310.zip', '/opt/anaconda3/envs/mlopsproj/lib/python3.10', '/opt/anaconda3/envs/mlopsproj/lib/python3.10/lib-dynload', '', '/Users/pravinut/mlopsproj_py/lib/python3.10/site-packages', '/Users/pravinut/vnit/sem2/MLOPS/Assignments/Assignment2/aiml_orchestrators_assignment/src', '/Users/pravinut/vnit/sem2/MLOPS/Assignments/Assignment2/aiml_orchestrators_assignment/src']


In [6]:
import numpy as np
import struct

def load_mnist_images(filename):
    filepath = os.path.join(DATA_PATH, filename)
    with open(filepath, 'rb') as f:
        magic, num, rows, cols = struct.unpack(">IIII", f.read(16))
        images = np.frombuffer(f.read(), dtype=np.uint8).reshape(num, 28, 28)
    return images

def load_mnist_labels(filename):
    filepath = os.path.join(DATA_PATH, filename)
    with open(filepath, 'rb') as f:
        magic, num = struct.unpack(">II", f.read(8))
        labels = np.frombuffer(f.read(), dtype=np.uint8)
    return labels

# Load the dataset
train_images = load_mnist_images("train-images.idx3-ubyte")
train_labels = load_mnist_labels("train-labels.idx1-ubyte")
test_images = load_mnist_images("t10k-images.idx3-ubyte")
test_labels = load_mnist_labels("t10k-labels.idx1-ubyte")

# Normalize images to range [0,1]
train_images = train_images / 255.0
test_images = test_images / 255.0

In [7]:
import tensorflow as tf
from tensorflow import keras

# Reshape the images for compatibility with TensorFlow
train_images = train_images.reshape(-1, 28, 28, 1)
test_images = test_images.reshape(-1, 28, 28, 1)

# Build a simple CNN model
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=5, validation_data=(test_images, test_labels))

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test Accuracy: {test_acc * 100:.2f}%")


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.9047 - loss: 0.3066 - val_accuracy: 0.9850 - val_loss: 0.0475
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.9870 - loss: 0.0446 - val_accuracy: 0.9870 - val_loss: 0.0405
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.9900 - loss: 0.0319 - val_accuracy: 0.9857 - val_loss: 0.0431
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.9929 - loss: 0.0220 - val_accuracy: 0.9886 - val_loss: 0.0358
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.9942 - loss: 0.0162 - val_accuracy: 0.9893 - val_loss: 0.0382
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9863 - loss: 0.0507 
Test Accuracy: 98.93%


In [8]:
model.save(os.path.join(MODEL_PATH, "mnist_model.h5"))
print("Model saved at:", os.path.join(MODEL_PATH, "mnist_model.h5"))

Model saved at: /Users/pravinut/vnit/sem2/MLOPS/Assignments/Assignment2/aiml_orchestrators_assignment/model/mnist_model.h5


In [9]:
import sys
import os
import numpy as np
import struct
import mlflow
import mlflow.tensorflow
import tensorflow as tf
from tensorflow import keras

# Set up paths
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATA_PATH = os.path.abspath(os.path.join(os.getcwd(), "../../data"))
MODEL_PATH = os.path.abspath(os.path.join(os.getcwd(), "../../model"))

os.makedirs(MODEL_PATH, exist_ok=True)

# Load MNIST dataset
def load_mnist_images(filename):
    filepath = os.path.join(DATA_PATH, filename)
    with open(filepath, 'rb') as f:
        magic, num, rows, cols = struct.unpack(">IIII", f.read(16))
        images = np.frombuffer(f.read(), dtype=np.uint8).reshape(num, 28, 28)
    return images

def load_mnist_labels(filename):
    filepath = os.path.join(DATA_PATH, filename)
    with open(filepath, 'rb') as f:
        magic, num = struct.unpack(">II", f.read(8))
        labels = np.frombuffer(f.read(), dtype=np.uint8)
    return labels

train_images = load_mnist_images("train-images.idx3-ubyte")
train_labels = load_mnist_labels("train-labels.idx1-ubyte")
test_images = load_mnist_images("t10k-images.idx3-ubyte")
test_labels = load_mnist_labels("t10k-labels.idx1-ubyte")

# Normalize data
train_images = train_images / 255.0
test_images = test_images / 255.0

# Reshape for TensorFlow
train_images = train_images.reshape(-1, 28, 28, 1)
test_images = test_images.reshape(-1, 28, 28, 1)

# MLflow Tracking URI (optional: use remote server)
mlflow.set_tracking_uri("http://localhost:5000")  # Change if using a remote MLflow server
mlflow.set_experiment("MNIST Experiment")

# Function to train model with different hyperparameters
def train_and_log_model(optimizer='adam', learning_rate=0.001, epochs=5, batch_size=64):
    with mlflow.start_run():
        mlflow.log_param("optimizer", optimizer)
        mlflow.log_param("learning_rate", learning_rate)
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("batch_size", batch_size)

        # Define model
        model = keras.Sequential([
            keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
            keras.layers.MaxPooling2D((2, 2)),
            keras.layers.Conv2D(64, (3, 3), activation='relu'),
            keras.layers.MaxPooling2D((2, 2)),
            keras.layers.Flatten(),
            keras.layers.Dense(64, activation='relu'),
            keras.layers.Dense(10, activation='softmax')
        ])

        # Compile model with different optimizers
        if optimizer == "adam":
            opt = keras.optimizers.Adam(learning_rate=learning_rate)
        elif optimizer == "sgd":
            opt = keras.optimizers.SGD(learning_rate=learning_rate)
        elif optimizer == "rmsprop":
            opt = keras.optimizers.RMSprop(learning_rate=learning_rate)
        else:
            raise ValueError("Unsupported optimizer!")

        model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

        # Train model
        history = model.fit(train_images, train_labels, 
                            epochs=epochs, 
                            batch_size=batch_size,
                            validation_data=(test_images, test_labels))

        # Evaluate model
        test_loss, test_acc = model.evaluate(test_images, test_labels)
        print(f"Test Accuracy: {test_acc * 100:.2f}%")

        # Log metrics
        mlflow.log_metric("test_accuracy", test_acc)
        mlflow.log_metric("test_loss", test_loss)

        # Save model
        model_path = os.path.join(MODEL_PATH, f"mnist_model_{optimizer}_{learning_rate}.h5")
        model.save(model_path)
        mlflow.tensorflow.log_model(model, artifact_path="model")

        print("Model saved at:", model_path)

# Run different experiments
train_and_log_model(optimizer="adam", learning_rate=0.001, epochs=5, batch_size=64)
train_and_log_model(optimizer="adam", learning_rate=0.0005, epochs=10, batch_size=64)
train_and_log_model(optimizer="sgd", learning_rate=0.01, epochs=5, batch_size=64)
train_and_log_model(optimizer="rmsprop", learning_rate=0.001, epochs=5, batch_size=64)


2025/02/25 11:52:31 INFO mlflow.tracking.fluent: Experiment with name 'MNIST Experiment' does not exist. Creating a new experiment.
/Users/pravinut/mlopsproj_py/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 13s 13ms/step - accuracy: 0.8754 - loss: 0.4095 - val_accuracy: 0.9841 - val_loss: 0.0498
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 12s 13ms/step - accuracy: 0.9828 - loss: 0.0531 - val_accuracy: 0.9867 - val_loss: 0.0393
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.9888 - loss: 0.0372 - val_accuracy: 0.9903 - val_loss: 0.0288
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.9917 - loss: 0.0273 - val_accuracy: 0.9881 - val_loss: 0.0374
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.9930 - loss: 0.0228 - val_accuracy: 0.9906 - val_loss: 0.0287
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9878 - loss: 0.0381 
Test Accuracy: 99.06%


2025/02/25 11:53:29 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2025/02/25 11:53:36 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model saved at: /Users/pravinut/vnit/sem2/MLOPS/Assignments/Assignment2/model/mnist_model_adam_0.001.h5
🏃 View run judicious-colt-708 at: http://localhost:5000/#/experiments/688274167480846939/runs/c32dc237e255476faa74615e71353d7c
🧪 View experiment at: http://localhost:5000/#/experiments/688274167480846939
Epoch 1/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 12s 12ms/step - accuracy: 0.8304 - loss: 0.5605 - val_accuracy: 0.9809 - val_loss: 0.0618
Epoch 2/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.9791 - loss: 0.0695 - val_accuracy: 0.9836 - val_loss: 0.0523
Epoch 3/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.9848 - loss: 0.0505 - val_accuracy: 0.9864 - val_loss: 0.0415
Epoch 4/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.9889 - loss: 0.0357 - val_accuracy: 0.9892 - val_loss: 0.0331
Epoch 5/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.9907 - loss: 0.0292 - val_accuracy: 0.9867 - val_loss: 0.0372
Epoch 6/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 11

2025/02/25 11:55:29 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2025/02/25 11:55:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model saved at: /Users/pravinut/vnit/sem2/MLOPS/Assignments/Assignment2/model/mnist_model_adam_0.0005.h5
🏃 View run bemused-duck-536 at: http://localhost:5000/#/experiments/688274167480846939/runs/25eb0ad3eaad4a29b0041bff580facde
🧪 View experiment at: http://localhost:5000/#/experiments/688274167480846939
Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 12s 13ms/step - accuracy: 0.5652 - loss: 1.5021 - val_accuracy: 0.8922 - val_loss: 0.3435
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.9172 - loss: 0.2732 - val_accuracy: 0.9471 - val_loss: 0.1755
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.9459 - loss: 0.1817 - val_accuracy: 0.9517 - val_loss: 0.1513
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.9597 - loss: 0.1335 - val_accuracy: 0.9660 - val_loss: 0.1091
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.9670 - loss: 0.1072 - val_accuracy: 0.9720 - val_loss: 0.0916
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accur

2025/02/25 11:56:31 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2025/02/25 11:56:36 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model saved at: /Users/pravinut/vnit/sem2/MLOPS/Assignments/Assignment2/model/mnist_model_sgd_0.01.h5
🏃 View run vaunted-koi-180 at: http://localhost:5000/#/experiments/688274167480846939/runs/ac12776c22e040a1b352fe27e16d5549
🧪 View experiment at: http://localhost:5000/#/experiments/688274167480846939
Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 13s 13ms/step - accuracy: 0.8782 - loss: 0.3976 - val_accuracy: 0.9851 - val_loss: 0.0490
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.9844 - loss: 0.0523 - val_accuracy: 0.9892 - val_loss: 0.0315
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.9897 - loss: 0.0342 - val_accuracy: 0.9894 - val_loss: 0.0322
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.9926 - loss: 0.0247 - val_accuracy: 0.9897 - val_loss: 0.0287
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.9940 - loss: 0.0201 - val_accuracy: 0.9896 - val_loss: 0.0305
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy:

2025/02/25 11:57:35 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2025/02/25 11:57:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model saved at: /Users/pravinut/vnit/sem2/MLOPS/Assignments/Assignment2/model/mnist_model_rmsprop_0.001.h5
🏃 View run youthful-rook-298 at: http://localhost:5000/#/experiments/688274167480846939/runs/f543ff7b860649c0a78b61b5d05df2d2
🧪 View experiment at: http://localhost:5000/#/experiments/688274167480846939


In [10]:
import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import mlflow
import mlflow.pytorch
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score

# Set paths
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATA_PATH = os.path.abspath(os.path.join(os.getcwd(), "../../data"))
MODEL_PATH = os.path.abspath(os.path.join(os.getcwd(), "../models"))
os.makedirs(MODEL_PATH, exist_ok=True)

# Load MNIST dataset
def load_mnist_images(filename):
    filepath = os.path.join(DATA_PATH, filename)
    with open(filepath, 'rb') as f:
        magic, num, rows, cols = struct.unpack(">IIII", f.read(16))
        images = np.frombuffer(f.read(), dtype=np.uint8).reshape(num, 28, 28)
    return images

def load_mnist_labels(filename):
    filepath = os.path.join(DATA_PATH, filename)
    with open(filepath, 'rb') as f:
        magic, num = struct.unpack(">II", f.read(8))
        labels = np.frombuffer(f.read(), dtype=np.uint8)
    return labels

train_images = load_mnist_images("train-images.idx3-ubyte")
train_labels = load_mnist_labels("train-labels.idx1-ubyte")
test_images = load_mnist_images("t10k-images.idx3-ubyte")
test_labels = load_mnist_labels("t10k-labels.idx1-ubyte")

# Normalize data
train_images = train_images / 255.0
test_images = test_images / 255.0

# Convert to PyTorch tensors
train_images_tensor = torch.tensor(train_images, dtype=torch.float32).unsqueeze(1)  # Add channel dim
train_labels_tensor = torch.tensor(train_labels, dtype=torch.long)
test_images_tensor = torch.tensor(test_images, dtype=torch.float32).unsqueeze(1)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.long)

# Create DataLoaders
train_dataset = TensorDataset(train_images_tensor, train_labels_tensor)
test_dataset = TensorDataset(test_images_tensor, test_labels_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define CNN Model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# MLflow setup
mlflow.set_tracking_uri("http://localhost:5000")  # Change for remote MLflow server
mlflow.set_experiment("MNIST Experiment PyTorch")

# Function to train model
def train_and_log_model(optimizer_type='adam', learning_rate=0.001, epochs=5, batch_size=64):
    with mlflow.start_run():
        mlflow.log_param("framework", "pytorch")
        mlflow.log_param("optimizer", optimizer_type)
        mlflow.log_param("learning_rate", learning_rate)
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("batch_size", batch_size)

        model = CNN()
        
        if optimizer_type == "adam":
            optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        elif optimizer_type == "sgd":
            optimizer = optim.SGD(model.parameters(), lr=learning_rate)
        elif optimizer_type == "rmsprop":
            optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)
        else:
            raise ValueError("Unsupported optimizer!")

        criterion = nn.CrossEntropyLoss()

        # Training Loop
        for epoch in range(epochs):
            for images, labels in train_loader:
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

            print(f"Epoch {epoch+1}, Loss: {loss.item()}")

        # Evaluate Model
        correct, total = 0, 0
        y_preds, y_trues = [], []

        with torch.no_grad():
            for images, labels in test_loader:
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                y_preds.extend(predicted.numpy())
                y_trues.extend(labels.numpy())

        accuracy = accuracy_score(y_trues, y_preds)
        print(f"Test Accuracy: {accuracy:.4f}")

        # Log results
        mlflow.log_metric("test_accuracy", accuracy)
        mlflow.pytorch.log_model(model, "model")

        # Save model
        model_path = os.path.join(MODEL_PATH, f"mnist_model_pytorch_{optimizer_type}_{learning_rate}.pth")
        torch.save(model.state_dict(), model_path)
        print("Model saved at:", model_path)

# Run different experiments
train_and_log_model(optimizer_type="adam", learning_rate=0.001, epochs=5, batch_size=64)
train_and_log_model(optimizer_type="adam", learning_rate=0.0005, epochs=10, batch_size=64)
train_and_log_model(optimizer_type="sgd", learning_rate=0.01, epochs=5, batch_size=64)
train_and_log_model(optimizer_type="rmsprop", learning_rate=0.001, epochs=5, batch_size=64)


2025/02/25 12:02:48 INFO mlflow.tracking.fluent: Experiment with name 'MNIST Experiment PyTorch' does not exist. Creating a new experiment.


Epoch 1, Loss: 0.040879782289266586
Epoch 2, Loss: 0.05770434811711311
Epoch 3, Loss: 0.009696338325738907
Epoch 4, Loss: 0.0024149278178811073
Epoch 5, Loss: 0.02982061356306076
Test Accuracy: 0.9905


2025/02/25 12:05:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model saved at: /Users/pravinut/vnit/sem2/MLOPS/Assignments/Assignment2/model/mnist_model_pytorch_adam_0.001.pth
🏃 View run invincible-goat-264 at: http://localhost:5000/#/experiments/917714700599549108/runs/4173e2e68e634767be47377a7a0ee8a9
🧪 View experiment at: http://localhost:5000/#/experiments/917714700599549108
Epoch 1, Loss: 0.056410931050777435
Epoch 2, Loss: 0.011537907645106316
Epoch 3, Loss: 0.012936309911310673
Epoch 4, Loss: 0.10841992497444153
Epoch 5, Loss: 0.005705532617866993
Epoch 6, Loss: 0.0024919300340116024
Epoch 7, Loss: 0.06047149747610092
Epoch 8, Loss: 0.014097253791987896
Epoch 9, Loss: 0.00027870299527421594
Epoch 10, Loss: 0.0018808655440807343
Test Accuracy: 0.9902


2025/02/25 12:09:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model saved at: /Users/pravinut/vnit/sem2/MLOPS/Assignments/Assignment2/model/mnist_model_pytorch_adam_0.0005.pth
🏃 View run flawless-loon-704 at: http://localhost:5000/#/experiments/917714700599549108/runs/a6cca298bd1f498c81e2540b4b4a9041
🧪 View experiment at: http://localhost:5000/#/experiments/917714700599549108
Epoch 1, Loss: 0.4543495178222656
Epoch 2, Loss: 0.3479456305503845
Epoch 3, Loss: 0.03886400908231735
Epoch 4, Loss: 0.14029084146022797
Epoch 5, Loss: 0.08178677409887314
Test Accuracy: 0.9685


2025/02/25 12:12:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model saved at: /Users/pravinut/vnit/sem2/MLOPS/Assignments/Assignment2/model/mnist_model_pytorch_sgd_0.01.pth
🏃 View run auspicious-fawn-444 at: http://localhost:5000/#/experiments/917714700599549108/runs/de5928e6c71d455e9548c3d4a4b46864
🧪 View experiment at: http://localhost:5000/#/experiments/917714700599549108
Epoch 1, Loss: 0.12048418819904327
Epoch 2, Loss: 0.1756865233182907
Epoch 3, Loss: 0.0034798074048012495
Epoch 4, Loss: 0.06390583515167236
Epoch 5, Loss: 0.017232922837138176
Test Accuracy: 0.9881


2025/02/25 12:14:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model saved at: /Users/pravinut/vnit/sem2/MLOPS/Assignments/Assignment2/model/mnist_model_pytorch_rmsprop_0.001.pth
🏃 View run intelligent-duck-421 at: http://localhost:5000/#/experiments/917714700599549108/runs/b117ff4027614fb89fb804d19c491ebf
🧪 View experiment at: http://localhost:5000/#/experiments/917714700599549108


In [9]:
import os
import sys
import struct
import torch
import torch.nn as nn
import torch.optim as optim
import mlflow
import mlflow.pytorch
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score

# Set paths
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATA_PATH = os.path.abspath(os.path.join(os.getcwd(), "../../data"))
MODEL_PATH = os.path.abspath(os.path.join(os.getcwd(), "../models"))
os.makedirs(MODEL_PATH, exist_ok=True)

# Load MNIST dataset
def load_mnist_images(filename):
    filepath = os.path.join(DATA_PATH, filename)
    with open(filepath, 'rb') as f:
        magic, num, rows, cols = struct.unpack(">IIII", f.read(16))
        images = np.frombuffer(f.read(), dtype=np.uint8).reshape(num, 1, 28, 28)  # Ensure correct shape
    return images

def load_mnist_labels(filename):
    filepath = os.path.join(DATA_PATH, filename)
    with open(filepath, 'rb') as f:
        magic, num = struct.unpack(">II", f.read(8))
        labels = np.frombuffer(f.read(), dtype=np.uint8)
    return labels

train_images = load_mnist_images("train-images.idx3-ubyte")
train_labels = load_mnist_labels("train-labels.idx1-ubyte")
test_images = load_mnist_images("t10k-images.idx3-ubyte")
test_labels = load_mnist_labels("t10k-labels.idx1-ubyte")

# Normalize data
train_images = train_images / 255.0
test_images = test_images / 255.0

# Convert to PyTorch tensors
train_images_tensor = torch.tensor(train_images, dtype=torch.float32)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.long)
test_images_tensor = torch.tensor(test_images, dtype=torch.float32)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.long)

# Create DataLoaders
train_dataset = TensorDataset(train_images_tensor, train_labels_tensor)
test_dataset = TensorDataset(test_images_tensor, test_labels_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define CNN Model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 64)
        self.fc2 = nn.Linear(64, 10)
    
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# MLflow setup
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("MNIST Experiment PyTorch")

# Function to train model
def train_and_log_model(optimizer_type='adam', learning_rate=0.001, epochs=5, batch_size=64):
    with mlflow.start_run():
        mlflow.log_param("framework", "pytorch")
        mlflow.log_param("optimizer", optimizer_type)
        mlflow.log_param("learning_rate", learning_rate)
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("batch_size", batch_size)

        model = CNN()
        
        if optimizer_type == "adam":
            optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        elif optimizer_type == "sgd":
            optimizer = optim.SGD(model.parameters(), lr=learning_rate)
        elif optimizer_type == "rmsprop":
            optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)
        else:
            raise ValueError("Unsupported optimizer!")

        criterion = nn.CrossEntropyLoss()

        # Training Loop
        for epoch in range(epochs):
            for images, labels in train_loader:
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

            print(f"Epoch {epoch+1}, Loss: {loss.item()}")

        # Evaluate Model
        correct, total = 0, 0
        y_preds, y_trues = [], []

        with torch.no_grad():
            for images, labels in test_loader:
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                y_preds.extend(predicted.numpy())
                y_trues.extend(labels.numpy())

        accuracy = accuracy_score(y_trues, y_preds)
        print(f"Test Accuracy: {accuracy:.4f}")

        # Log results
        mlflow.log_metric("test_accuracy", accuracy)
        mlflow.pytorch.log_model(model, "model")

        # Save model
        model_path = os.path.join(MODEL_PATH, f"mnist_model_pytorch_{optimizer_type}_{learning_rate}.pth")
        torch.save(model.state_dict(), model_path)
        print("Model saved at:", model_path)

# Run different experiments
train_and_log_model(optimizer_type="adam", learning_rate=0.001, epochs=5, batch_size=64)
train_and_log_model(optimizer_type="adam", learning_rate=0.0005, epochs=10, batch_size=64)
train_and_log_model(optimizer_type="sgd", learning_rate=0.01, epochs=5, batch_size=64)
train_and_log_model(optimizer_type="rmsprop", learning_rate=0.001, epochs=5, batch_size=64)

Epoch 1, Loss: 0.03241514414548874
Epoch 2, Loss: 0.014767859131097794
Epoch 3, Loss: 0.030488135293126106
Epoch 4, Loss: 0.0010689987102523446
Epoch 5, Loss: 0.0004080650396645069
Test Accuracy: 0.9882


2025/02/25 14:57:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model saved at: /Users/pravinut/vnit/sem2/MLOPS/Assignments/Assignment2/aiml_orchestrators_assignment/models/mnist_model_pytorch_adam_0.001.pth
🏃 View run rare-hound-94 at: http://localhost:5000/#/experiments/917714700599549108/runs/e442d624303a471385d90aaf59d76dcd
🧪 View experiment at: http://localhost:5000/#/experiments/917714700599549108
Epoch 1, Loss: 0.056003641337156296
Epoch 2, Loss: 0.02509920299053192
Epoch 3, Loss: 0.0037948626559227705
Epoch 4, Loss: 0.020609175786376
Epoch 5, Loss: 0.0011451035970821977
Epoch 6, Loss: 0.009010781534016132
Epoch 7, Loss: 0.10222824662923813
Epoch 8, Loss: 0.002100937068462372
Epoch 9, Loss: 0.01915203034877777
Epoch 10, Loss: 0.0058694081380963326
Test Accuracy: 0.9886


2025/02/25 15:01:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model saved at: /Users/pravinut/vnit/sem2/MLOPS/Assignments/Assignment2/aiml_orchestrators_assignment/models/mnist_model_pytorch_adam_0.0005.pth
🏃 View run thundering-perch-198 at: http://localhost:5000/#/experiments/917714700599549108/runs/dc9d8b1f65d54f6f82f89e7a16b637ca
🧪 View experiment at: http://localhost:5000/#/experiments/917714700599549108
Epoch 1, Loss: 0.3711227476596832
Epoch 2, Loss: 0.2263508439064026
Epoch 3, Loss: 0.29550421237945557
Epoch 4, Loss: 0.2700895369052887
Epoch 5, Loss: 0.22489100694656372
Test Accuracy: 0.9669


2025/02/25 15:02:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model saved at: /Users/pravinut/vnit/sem2/MLOPS/Assignments/Assignment2/aiml_orchestrators_assignment/models/mnist_model_pytorch_sgd_0.01.pth
🏃 View run rogue-cod-233 at: http://localhost:5000/#/experiments/917714700599549108/runs/ce88e10e2f234ea79e7d4e22f3d63b7b
🧪 View experiment at: http://localhost:5000/#/experiments/917714700599549108
Epoch 1, Loss: 0.046352457255125046
Epoch 2, Loss: 0.016106197610497475
Epoch 3, Loss: 0.013561534695327282
Epoch 4, Loss: 0.006273791193962097
Epoch 5, Loss: 0.0036959885619580746
Test Accuracy: 0.9890


2025/02/25 15:04:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model saved at: /Users/pravinut/vnit/sem2/MLOPS/Assignments/Assignment2/aiml_orchestrators_assignment/models/mnist_model_pytorch_rmsprop_0.001.pth
🏃 View run hilarious-eel-781 at: http://localhost:5000/#/experiments/917714700599549108/runs/4faf36d8ab8a48d1967075966a7bb631
🧪 View experiment at: http://localhost:5000/#/experiments/917714700599549108
